## Capstone Project - The Battle of the Neighborhood

In [1]:
import time
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


### 1. Introduction Section: 

### Scenario

I recently moved to San Jose, California. Looking for afordable good neighboorhood to rent near downtown San Jose. he apartment in San Jose  must meet the following demands:

- apartment must be 2 or 3 bedrooms
- desired location is near a rail or cal station and within 1.0 mile (1.6 km) radius
- price of rent not exceed $7,000 per month
- top ammenities in the selected neighborhood shall be similar to current residence
- desirable to have venues such as coffee shops, Asian restaurants, gym and food shops


#### Business Problem 

The challenge is to find a suitable apartment for rent in San Jose California that complies with the demands on location, price and venues. The data required to resolve this challenge is described in the following section 2, below.

### Interested Audience

I believe this is a relevant challenge with valid questions for anyone moving to other large city. The methodology  is also applicable for anyone interested in exploring starting or locating a new business in any city. Lastly, it can also serve as a good practical exercise to develop Data Science skills.

### 2. Data Section

Description of the Data:

The following data is required to answer the issues of the problem:

- List of Boroughs and neighborhoods of San Jose with their geodata (latitude and longitude)
- List of rail and cal stations in San Jose with their address location
- List of apartments for rent in San Jose area with their addresses and price
- Preferably, a list of apartment for rent with additional information, such as price, address, area, # of beds, etc
- Venues for each San Jose neighborhood ( than can be clustered)
- Venues for rail stations, as needed

### How the data will be used to solve the problem

The data will be used as follows:

- Use Foursquare and geopy data to map top 10 venues for all  San Jose neighborhoods and clustered in groups
- Use foursquare and geopy data to map the location of rail or cal stations , separately and on top of the above clustered map in order to be able to identify the venues and ammenities near each  station, or explore each station separately
- Use Foursquare and geopy data to map the location of rental places, in some form, linked to the rail locations.
create a map that depicts, for instance, the average rental price per square ft, around a radious of 3.0 mile around each rail station - or a similar metrics. 
Addresses from rental locations will be converted to geodata( lat, long) using Geopy-distance.

The procesing of these DATA will allow to answer the key questions to make a decision:

- what is the cost of rent (per square ft) around a mile radius from each subway metro station?
- what is the area of Manhattan with best rental pricing that meets criteria established?
- What are the venues of the two best places to live? How the prices compare?
- How venues distribute among San Jose neighborhoods and around metro stations?
- Are there tradeoffs between size and price and location?
- Any other interesting statistical data findings of the real estate and overall data.

### Mapping of San Jose, California

In [2]:
address = 'Downtown San Jose, California'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of San Jose home are {}, {}.'.format(latitude, longitude))

/Users/bnakarmi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of San Jose home are 37.3359104, -121.8910758.


In [3]:

neighborhood_latitude=37.3359104
neighborhood_longitude=-121.8910758

In [1]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


In [5]:

LIMIT = 1500 # limit of number of venues returned by Foursquare API
radius = 40000 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=VK1AKOSSMHML0TSP30JU55E4TKK0K0LWXLAR12ZZD0M3MNXR&client_secret=BIM0VEVUXVUHVCNDTH41WIJ50MOXMBOXLZBBMESKOWJ0JD0S&v=20180605&ll=37.3359104,-121.8910758&radius=40000&limit=1500'

In [2]:
results = requests.get(url).json()


NameError: name 'requests' is not defined

In [7]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [8]:
venues = results['response']['groups'][0]['items']
    
SJnearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories','venue.location.lat', 'venue.location.lng','venue.location.postalCode',
                    'venue.location.city']
SJnearby_venues =SJnearby_venues.loc[:, filtered_columns]

# filter the category for each row
SJnearby_venues['venue.categories'] = SJnearby_venues.apply(get_category_type, axis=1)

# clean columns
SJnearby_venues.columns = [col.split(".")[-1] for col in SJnearby_venues.columns]

SJnearby_venues

,name,categories,lat,lng,postalCode,city
0,San Pedro Square Market,Food Court,37.336480,-121.894403,95110,San Jose
1,Whole Foods Market,Grocery Store,37.332086,-121.904623,95126,San Jose
2,The Studio Climbing & Fitness,Climbing Gym,37.330273,-121.886095,95112,San Jose
3,Back A Yard Caribbean American Grill,Caribbean Restaurant,37.336683,-121.892749,95113,San Jose
4,Silicon Valley Capital Club,Restaurant,37.333945,-121.889317,95113,San Jose
5,Luna Mexican Restaurant,Mexican Restaurant,37.333935,-121.915180,95126,San Jose
6,California Theatre,Opera House,37.330659,-121.887356,95113,San Jose
7,Trader Joe's,Grocery Store,37.340948,-121.909405,95110,San Jose
8,Yogurtland,Frozen Yogurt Shop,37.333020,-121.884331,95112,San Jose
9,Poor House Bistro,Cajun / Creole Restaurant,37.329650,-121.900431,95110,San Jose


In [9]:
##number of unique categories
print('There are {} unique categories.'.format(len(SJnearby_venues['categories'].unique())))
#print(SJnearby_venues[['categories']])
#frquency of each category
print(SJnearby_venues['categories'].value_counts())

There are 52 unique categories.
Park                         9
Grocery Store                8
Sandwich Place               6
Coffee Shop                  5
Gym                          5
Trail                        5
American Restaurant          3
Mexican Restaurant           3
Pizza Place                  3
Wine Shop                    3
Bakery                       3
Ice Cream Shop               2
Café                         2
Japanese Restaurant          2
Burger Joint                 2
Pharmacy                     2
Climbing Gym                 2
Mediterranean Restaurant     1
Sushi Restaurant             1
Music Venue                  1
Peruvian Restaurant          1
Deli / Bodega                1
Fast Food Restaurant         1
Opera House                  1
Cajun / Creole Restaurant    1
Smoke Shop                   1
Asian Restaurant             1
Cosmetics Shop               1
Greek Restaurant             1
Caribbean Restaurant         1
Shopping Mall                1
Restaur

In [133]:
#city 
conm = SJnearby_venues.groupby(['city']).count()
conm

,name,categories,lat,lng,postalCode
city,,,,,
Campbell,5,5,5,5,5
Cupertino,10,10,10,10,10
Los Altos,1,1,1,1,1
Los Gatos,11,11,11,11,10
Milpitas,1,1,1,1,1
Mountain View,4,4,4,4,4
San Jose,46,46,46,46,46
Santa Clara,7,7,7,7,7
Saratoga,5,5,5,5,5


### Map of SAN JOSE with venues near residence place - for reference

In [10]:
# create map of SAN JOSE place  using latitude and longitude values
map_sj = folium.Map(location=[latitude, longitude], zoom_start=20)

# add markers to map
for lat, lng, label in zip(SJnearby_venues['lat'], SJnearby_venues['lng'], SJnearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.RegularPolygonMarker(
        [lat, lng],
        number_of_sides=4,
        radius=10,
        popup=label,
        color='blue',
        fill_color='#0f0f0f',
        fill_opacity=0.7,
    ).add_to(map_sj)  
    
map_sj